## run this cell before starting spark cluster connection (corrects module error)

In [6]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [9]:
df_ps = spark.read.json("/user/sprenner/training_official_april.json")

### Create temporary table view

In [10]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="hits", outputCol="bucketedFeatures")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

In [11]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucket \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0    425303
1               1.0    351983

In [12]:
final_data = bucketedData.withColumn("class_bucket", when(bucketedData["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))

In [13]:
final_data.createOrReplaceTempView("final")
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    351983
1      nocache    425303

In [14]:
final_data.describe().toPandas()

summary  account          accountHash         clientState  \
0   count   777286               777286              777286   
1    mean     None  1.935684591732567E8                None   
2  stddev     None  1.136131724108707E9                None   
3     min  aagaard          -2127257360  CannotAuthenticate   
4     max   zzheng           2143592175      SourceNotFound   

        clientStateHash                                            dataset  \
0                777286                                             777286   
1   -9.74016245191591E8                                               None   
2  2.6212986830087784E8                                               None   
3           -1922357674  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...   
4            1269532611  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0                777286       777286                777286          777286   
1  1.4554549800768314E7         None  -7.552649492442358E8            None   
2  1.1941649925847795E9         None     8.3148655014496E8            None   
3           -2147401420    data10_hi           -2057006107     c1130_m1847   
4            2147445885   mc16_valid            2109319881      x557_m1947   

    dataset_versionHash datatype          datatypeHash eventType  \
0                777286   777286                777286    777286   
1   2.973310495739393E8     None   4.947060556405943E8      None   
2  1.2288188131163657E9     None  1.1700303292632794E9      None   
3           -2146716518      AOD           -2070622869  download   
4            2146405934      log            2124512252  download   

  eventTypeHash eventVersion      eventVersionHash  \
0        777286       777286                777286   
1    5.937808E7         None  -1.992468944589716E9   
2           0.0         None   3.073273964746104E8   
3      59378080       1.13.2           -2044360925   
4      59378080       1.15.5            1261702107   

                             filename         filenameHash  \
0                              777286               777286   
1                                None  -248557.46481861244   
2                                None  1.240265814971078E9   
3    AOD.05371635._000001.pool.root.1          -2147478094   
4  log.13787275._000072.job.log.tgz.1           2147482617   

       filesize_double                hits                  hostname  \
0               777286              777286                    777286   
1   6.11402028756827E8  2.8777927300890536                      None   
2  1.240877180994578E9  3.9916293684162323                      None   
3                304.0                   1    ReCaS-ui-01.cs.infn.it   
4      1.1991446515E10                  48  xenia.nevis.columbia.edu   

            hostnameHash prod_step          prod_stepHash     project  \
0                 777286    777286                 777286      777286   
1  -1.1804844919942595E8      None  -3.3290311308177426E7        None   
2   1.2286452227051144E9      None   1.1050073037449553E9        None   
3            -2133418459  PhysCont            -1329401808   data10_hi   
4             2129595856     simul             2037678380  mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0                777286   777286                777286                 777286   
1  -7.551860864380562E8     None  3.5590511037907284E8                   None   
2   8.315047181989846E8     None  1.0536208601928813E9                   None   
3           -2057006107     davs           -1607963002        AGLT2_CALIBDISK   
4            2109319881      srm            2128034789  ZA-WITS-CORE_PRODDISK   

         remoteSiteHash         run_number        run_numberHash       scope  \
0                777286             777286                777286      777286   
1  -6.188806654015021E8  349501.9749693951    9.53754871671

## Bucketing and Hashing End

In [15]:

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator




assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
                                       "eventVersionHash","filenameHash","filesize_double","hostnameHash","protocolHash",
                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash","traceTimeentryUnix",
                                       "transferStart","uuidHash",
                                       "projectHash", "run_numberHash", "stream_nameHash", "prod_stepHash", "datatypeHash",
                                       "dataset_versionHash"], outputCol="features")
output = assembler.transform(final_data)
#output = output.select("bucketedFeatures", "features")
        
train_data, test_data = output.randomSplit([.8,.2],seed=1234)
rf = RandomForestClassifier(labelCol='bucketedFeatures', featuresCol='features',numTrees=300, seed=1234)
model = rf.fit(train_data)
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedFeatures", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0970723


In [16]:
model.featureImportances

SparseVector(23, {0: 0.089, 1: 0.38, 2: 0.0039, 3: 0.0387, 5: 0.002, 6: 0.0, 7: 0.1425, 8: 0.0147, 9: 0.0194, 10: 0.0085, 11: 0.0324, 12: 0.0, 13: 0.0261, 14: 0.0219, 15: 0.0245, 16: 0.0081, 17: 0.0369, 18: 0.0048, 19: 0.0082, 20: 0.0273, 21: 0.0755, 22: 0.0353})

## predict downloads for may with trained model

In [81]:
df_mai = spark.read.json("/user/sprenner/training_official_mai.json")

In [82]:
df_mai.describe().toPandas()

summary   account           accountHash     clientState  \
0   count   5316524               5316524         5316524   
1    mean      None   5.676034578304038E7            None   
2  stddev      None  1.0782381847434077E9            None   
3     min  aalfonsi           -2144719344  AttributeError   
4     max    zzheng            2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2   3.393227998953656E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

           datasetHash datasetScope      datasetScopeHash eventType  \
0              5316524      5316524               5316524   5316524   
1  8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.264352081440324E9         None   1.159227636261917E9      None   
3          -2147477854    cond09_mc           -2145215727  download   
4           2147445885       valid2            2144116684  download   

  eventTypeHash eventVersion       eventVersionHash  \
0       5316524      5316524                5316524   
1    5.937808E7         None  -2.0386347870917916E9   
2           0.0         None   1.2667515763379405E8   
3      59378080       1.13.2            -2044360925   
4      59378080       1.16.1             1352846300   

                                            filename         filenameHash  \
0                                            5316524              5316524   
1                                               None   -576003.5649204254   
2                                               None  1.238741621683768E9   
3                                              0.dat          -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...           2147479805   

        filesize_double                hits                  hostname  \
0               5316524             5316524                   5316524   
1  2.8167336660135007E8   6.517046664324284                      None   
2  1.0094883391407572E9  13.920048576048558                      None   
3                   2.0                   1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                 249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325292E9     None  9.234807216961868E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None   1.243718578580587E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2      1.1593529202301E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

            traceIdHash                    traceIp            traceIpHash  \
0               5316524                    5316524                5316524   
1   -254286.60280984343                       None  -4.1963757976758495E7   
2  1.2400714422504258E9                       None   1.2010453269587853E9   
3           -2147483583  2001:1458:201:e4::100:28c            -2143850866   
4            2147482

In [83]:
assembler_mai = VectorAssembler(
    inputCols=["traceTimeentryUnix", "transferStart", "accountHash","clientStateHash","datasetHash", "datasetScopeHash",
               "eventTypeHash", "eventVersionHash","filenameHash", "hostnameHash", "protocolHash", "remoteSiteHash", 
               "scopeHash", "traceIdHash", "traceIpHash", "uuidHash", "filesize_double",
               "projectHash", "run_numberHash", "stream_nameHash", "prod_stepHash", "datatypeHash",
               "dataset_versionHash"],
    outputCol="features")

output_mai = assembler_mai.transform(df_mai)

In [84]:
predicted_mai = model.transform(output_mai)

In [85]:
predicted_mai=predicted_mai.withColumn("filesize", predicted_mai["filesize_double"].cast(IntegerType()))

In [86]:
predicted_mai = predicted_mai.drop('filesize_double')

## save separately for plotting

In [87]:
df_mai_plot = predicted_mai.select('hits', 'prediction')
df_mai_plot.write.csv('/user/sprenner/forest_plot_mai_official_predictions_numTrees300_seed1234_cache01.csv', sep='\t')

In [88]:
df_mai_sim = predicted_mai.select('uuid', 'account', 'dataset', 'filename', 'eventType', 'clientState', 
                                  from_unixtime(predicted_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction').sort(col("day").asc())

In [89]:
df_mai_sim.limit(5).toPandas()

uuid account  \
0  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
1  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
2  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
3  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
4  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   

                                             dataset  \
0  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
1  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
2  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
3  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
4  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   

                           filename eventType         clientState         day  \
0  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
1  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
2  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
3  AOD.12911492._000332.pool.root.1  download  ServiceUnavailable  2018-05-01   
4  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   

   traceTimeentryUnix    filesize  prediction  
0        1.525170e+09  2147483647         1.0  
1        1.525170e+09  2147483647         1.0  
2        1.525170e+09  2147483647         1.0  
3        1.525170e+09  2147483647         1.0  
4        1.525170e+09  2147483647         1.0

In [90]:
df_mai_sim.write.csv('/user/sprenner/forest_mai_official_predictions_numTrees300_seed1234_cache01.csv', sep='\t')